In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

#### Few-shot learning

In [4]:
from langchain import PromptTemplate
from langchain import FewShotPromptTemplate

# create our examples
examples = [
    {
        "query": "What's the weather like?",
        "answer": "It's raining cats and dogs, better bring an umbrella!"
    }, 
    {
        "query": "How old are you?",
        "answer": "Age is just a number, but I'm timeless."
    }
]

# create an example template
example_template = """
User: {query}
AI: {answer}
"""

# create a prompt example from above template
example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

# now break our previous prompt into a prefix and suffix
# the prefix is our instructions
prefix = """The following are excerpts from conversations with an AI assistant. 
The assistant is known for its humor and wit, providing entertaining and amusing responses to users' questions. 
Here are some examples:
"""
# and the suffix our user input and output indicator
suffix = """
User: {query}
AI: """

# now create the few-shot prompt template
few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)

In [5]:
# After creating a template, we pass the example and user query, and we get the results

from langchain.chat_models import ChatOpenAI
from langchain import LLMChain

# load the model
chat = ChatOpenAI(model_name="gpt-4", temperature=0.0)

chain = LLMChain(llm=chat, prompt=few_shot_prompt_template)
chain.run("What's the meaning of life?")

"The meaning of life? Well, I believe it's a jigsaw puzzle. You have to assemble it yourself and everyone's picture looks different. But remember, I'm an AI, I don't even have a life to begin with!"

#### Text Summarization

In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [8]:
# this could generalize to other tasks, not just summarization

summarization_template = "Summarize the following text to one sentence: {text}"
summarization_prompt = PromptTemplate(
    input_variables=["text"], 
    template=summarization_template
)
summarization_chain = LLMChain(llm=llm, prompt=summarization_prompt)

In [9]:
text = """
LangChain provides many modules that can be used to build language model applications. Modules can be combined to create more complex applications, or be used individually for simple applications. 
The most basic building block of LangChain is calling an LLM on some input. Let’s walk through a simple example of how to do this. For this purpose, let’s pretend we are building a service that generates a company name based on what the company makes."""

summarized_text = summarization_chain.predict(text=text)

In [10]:
summarized_text

'LangChain offers various modules for building language model applications, which can be used individually or combined to create more complex applications, such as generating company names based on their products.'

#### Simple QA chain example

In [12]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import OpenAI

template = """
Question: {question}
Answer: """

prompt = PromptTemplate(
    template=template,
    input_variables=["question"]
)

llm = OpenAI(model_name="text-davinci-003", temperature=0)
chain = LLMChain(llm=llm, prompt=prompt)

# We define a custom prompt template by creating an instance of the PromptTemplate class. 
# The template string contains a placeholder {question} for the input question, followed by the "Answer:" label (on new line).  
# The input_variables argument is set to the list of available placeholders in the prompt (like a question in this case) to indicate the name of the variable that the chain will replace in the template.run() method.

chain.run("what is the meaning of life?")


# Here is an alternative format with placeholder for the product (that is subject to the question) and not the whole question:
# prompt = PromptTemplate(
#   input_variables=["product"],
#   template="What is a good name for a company that makes {product}?",
# )
# chain.run("wireless headphones")

' The meaning of life is subjective and can vary from person to person. For some, it may be to find happiness and fulfillment, while for others it may be to make a difference in the world. Ultimately, the meaning of life is up to each individual to decide.'

#### Chat Models

Chat Message Types

**SystemMessage**: These messages provide initial instructions, context, or data for the AI model. They set the objectives the AI should follow and can help in controlling the AI's behavior. System messages are not user inputs but rather guidelines for the AI to operate within.

**HumanMessage**: These messages come from the user and represent their input to the AI model. The AI model is expected to respond to these messages. In LangChain, you can customize the human prefix (e.g., "User") in the conversation summary to change how the human input is represented.

**AIMessage**: These messages are sent from the AI's perspective as it interacts with the human user. They represent the AI's responses to human input. Like **HumanMessage**, you can also customize the AI prefix (e.g., "AI Assistant" or "AI") in the conversation summary to change how the AI's responses are represented.

In [14]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
  HumanMessage,
  SystemMessage
)

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

messages = [
	SystemMessage(content="You are a helpful assistant that translates English to French."),
	HumanMessage(content="Translate the following sentence: I love programming.")
]

# We pass the list of messages to the chatbot using the chat()Copy function. 
# The chatbot processes the input messages, considers the context provided by the system message, and then translates the given English sentence into French. 

chat(messages)

AIMessage(content="J'adore la programmation.", additional_kwargs={}, example=False)

#### SystemMessagePromptTemplate and HumanMessagePromptTemplate
(the messages are part of a template, the human_template contains a placeholder which will be replaced by the actual subject of the qestion)

In [11]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

template = "You are an assistant that helps users find information about movies."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = "Find information about the movie {movie_title}."
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

response = chat(chat_prompt.format_prompt(movie_title="Inception").to_messages())

print(response.content)

# Using the to_messages object in LangChain allows you to convert the formatted value of a chat prompt template into a list of message objects. 
# This is useful when working with chat models, as it provides a structured way to manage the conversation and ensures that the chat model can understand the context and roles of the messages.

"Inception" is a science fiction action film directed by Christopher Nolan. It was released in 2010 and stars Leonardo DiCaprio, Joseph Gordon-Levitt, Ellen Page, Tom Hardy, and Marion Cotillard. The film follows a professional thief who steals information by infiltrating the subconscious of his targets through their dreams. 

The plot revolves around Dom Cobb (played by DiCaprio), who is offered a chance to have his criminal history erased in exchange for performing the act of "inception" - planting an idea in someone's mind rather than stealing it. As Cobb assembles a team to carry out this dangerous mission, they must navigate through various dream levels, each with its own unique challenges and risks.

"Inception" received critical acclaim for its originality, visual effects, and complex storytelling. It was praised for its thought-provoking themes and exploration of the nature of reality and dreams. The film was a commercial success, grossing over $828 million worldwide.

It won f